In [1]:
from Bio import SeqIO

In [2]:
from Bio.Blast import NCBIWWW

In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
import math
import pickle

In [4]:
df=pd.read_csv('../dataset/drug/structure links.csv')
df

,DrugBank ID,Name,CAS Number,Drug Groups,InChIKey,InChI,SMILES,Formula,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,ChEMBL ID,HET ID,ChemSpider ID,BindingDB ID
0,DB00006,Bivalirudin,128270-60-0,approved; investigational,OIRCOABEOLEUMC-GEJPAHFPSA-N,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33,NaN,D03136,16129704.0,46507415.0,59173.0,CHEMBL2103749,NaN,10482069.0,50248103.0
1,DB00007,Leuprolide,53714-56-0,approved; investigational,GFIJNRVAKGFPGQ-LIJARHBVSA-N,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12,C07612,D08113,NaN,46507635.0,6427.0,CHEMBL1201199,NaN,571356.0,50369395.0
2,DB00014,Goserelin,65807-02-5,approved,BLCLNMBMMGCOAS-URPVMXJPSA-N,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14,NaN,D00573,5311128.0,46507336.0,5523.0,CHEMBL1201247,NaN,4470656.0,NaN
3,DB00027,Gramicidin D,1405-97-6,approved,NDAYQJDHGXTBJL-MWWSRJDJSA-N,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16,NaN,D04369,45267103.0,46507412.0,NaN,CHEMBL557217,NaN,24623445.0,NaN
4,DB00035,Desmopressin,16679-58-6,approved,NFLWUMRGJYTJIN-PNIOQBSNSA-N,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2,C06944,D00291,NaN,NaN,4450.0,CHEMBL1429,NaN,4470602.0,50205308.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11829,DB16403,Isoquercitrin,21637-25-2,investigational,OPJZLUXFQFQYAI-GNPVFZCLSA-N,InChI=1S/C21H20O12/c22-6-12(27)19-16(29)17(30)...,[H][C@@]1(O[C@@H](OC2=C(OC3=C(C(O)=CC(O)=C3)C2...,C21H20O12,NaN,NaN,NaN,NaN,28299.0,NaN,NaN,4588355.0,NaN
11830,DB16404,Carvacrol,499-75-2,experimental,RECUKUPTGUEGMW-UHFFFAOYSA-N,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)10(11)6-9/h4...,CC(C)C1=CC(O)=C(C)C=C1,C10H14O,C09840,NaN,NaN,NaN,3440.0,CHEMBL281202,NaN,21105867.0,50240433.0
11831,DB16406,Subasumstat,1858276-04-6,investigational,LXRZVMYMQHNYJB-UNXOBOICSA-N,InChI=1S/C25H28ClN5O5S2/c1-13-18(23-19-7-16(26...,CC1=C(C=C(S1)C(=O)C1=C(N[C@H]2C[C@H](O)[C@@H](...,C25H28ClN5O5S2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72380106.0,NaN
11832,DB16407,Azvudine,1011529-10-4,investigational,YGDJSOUUXVJBBP-XZMZPDFPSA-N,"InChI=1S/C9H12FN6O4/c10-5-6(18)9(3-17,14-15-12...",NC1=NC(=O)N(C=C1)[C@@H]1O[C@@](CO)(N=[N]=N)[C@...,C9H12FN6O4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
newdf=df.drop(['CAS Number', 'InChIKey','InChI','KEGG Compound ID','KEGG Drug ID','PubChem Compound ID','PubChem Substance ID',
         'ChEBI ID','ChEMBL ID','HET ID','ChemSpider ID','BindingDB ID'], axis = 1) 
newdf

,DrugBank ID,Name,Drug Groups,SMILES,Formula
0,DB00006,Bivalirudin,approved; investigational,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33
1,DB00007,Leuprolide,approved; investigational,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12
2,DB00014,Goserelin,approved,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14
3,DB00027,Gramicidin D,approved,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16
4,DB00035,Desmopressin,approved,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2
...,...,...,...,...,...
11829,DB16403,Isoquercitrin,investigational,[H][C@@]1(O[C@@H](OC2=C(OC3=C(C(O)=CC(O)=C3)C2...,C21H20O12
11830,DB16404,Carvacrol,experimental,CC(C)C1=CC(O)=C(C)C=C1,C10H14O
11831,DB16406,Subasumstat,investigational,CC1=C(C=C(S1)C(=O)C1=C(N[C@H]2C[C@H](O)[C@@H](...,C25H28ClN5O5S2
11832,DB16407,Azvudine,investigational,NC1=NC(=O)N(C=C1)[C@@H]1O[C@@](CO)(N=[N]=N)[C@...,C9H12FN6O4


In [6]:
mf_obj=[]
mf_list=[]

In [7]:
def morgan_fingerprint_conversion(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        fp_morgan = AllChem.GetMorganFingerprint(mol,2)
        fp_morgan_hashed = AllChem.GetMorganFingerprintAsBitVect(mol,2,nBits=2048)
        mf_obj.append(fp_morgan_hashed)
        mf = fp_morgan_hashed.ToBitString()
        return mf
    except:
        mf_obj.append(np.nan)
        return np.nan

In [8]:
for i in range(len(newdf)):
    mf=''
    if pd.isnull(newdf.loc[i]['SMILES']):
        mf_list.append(np.nan)
        mf_obj.append(np.nan)
    else:
        mf = morgan_fingerprint_conversion(newdf.loc[i]['SMILES'])
        mf_list.append(mf)

In [10]:
len(mf_list)

11834

In [11]:
newdf['Morgan Fingerprint']=mf_list
newdf['Morgan object']=mf_obj

In [12]:
newdf

,DrugBank ID,Name,Drug Groups,SMILES,Formula,Morgan Fingerprint,Morgan object
0,DB00006,Bivalirudin,approved; investigational,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33,0100000000000000000000000000000000000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,DB00007,Leuprolide,approved; investigational,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,DB00014,Goserelin,approved,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,DB00027,Gramicidin D,approved,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16,0100000000000000000000000000000001000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,DB00035,Desmopressin,approved,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
11829,DB16403,Isoquercitrin,investigational,[H][C@@]1(O[C@@H](OC2=C(OC3=C(C(O)=CC(O)=C3)C2...,C21H20O12,0100000000000000000000000000000000001000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11830,DB16404,Carvacrol,experimental,CC(C)C1=CC(O)=C(C)C=C1,C10H14O,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11831,DB16406,Subasumstat,investigational,CC1=C(C=C(S1)C(=O)C1=C(N[C@H]2C[C@H](O)[C@@H](...,C25H28ClN5O5S2,0000000001000000000010000000000100000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
11832,DB16407,Azvudine,investigational,NC1=NC(=O)N(C=C1)[C@@H]1O[C@@](CO)(N=[N]=N)[C@...,C9H12FN6O4,NaN,NaN


In [13]:
newdf.to_pickle("../dataset/drug/drug.pkl")

In [14]:
newdf["Morgan object"][0].ToBitString()
newdf["Morgan object"]

0        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                               ...                        
11829    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
11830    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
11831    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
11832                                                  NaN
11833    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Morgan object, Length: 11834, dtype: object

In [78]:
#removing unwanted

In [43]:
drugdf=pd.read_pickle("../dataset/drug/drug.pkl")
drugdf

,DrugBank ID,Name,Drug Groups,SMILES,Formula,Morgan Fingerprint,Morgan object
0,DB00006,Bivalirudin,approved; investigational,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33,0100000000000000000000000000000000000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,DB00007,Leuprolide,approved; investigational,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,DB00014,Goserelin,approved,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,DB00027,Gramicidin D,approved,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16,0100000000000000000000000000000001000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,DB00035,Desmopressin,approved,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
11829,DB16403,Isoquercitrin,investigational,[H][C@@]1(O[C@@H](OC2=C(OC3=C(C(O)=CC(O)=C3)C2...,C21H20O12,0100000000000000000000000000000000001000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11830,DB16404,Carvacrol,experimental,CC(C)C1=CC(O)=C(C)C=C1,C10H14O,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11831,DB16406,Subasumstat,investigational,CC1=C(C=C(S1)C(=O)C1=C(N[C@H]2C[C@H](O)[C@@H](...,C25H28ClN5O5S2,0000000001000000000010000000000100000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
11832,DB16407,Azvudine,investigational,NC1=NC(=O)N(C=C1)[C@@H]1O[C@@](CO)(N=[N]=N)[C@...,C9H12FN6O4,NaN,NaN


In [44]:
drugdf = drugdf[drugdf['Morgan Fingerprint'].notna()]
drugdf

,DrugBank ID,Name,Drug Groups,SMILES,Formula,Morgan Fingerprint,Morgan object
0,DB00006,Bivalirudin,approved; investigational,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33,0100000000000000000000000000000000000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,DB00007,Leuprolide,approved; investigational,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,DB00014,Goserelin,approved,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,DB00027,Gramicidin D,approved,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16,0100000000000000000000000000000001000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,DB00035,Desmopressin,approved,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
11828,DB16400,Sibofimloc,investigational,[H][C@@]1(O[C@H](CO)[C@@H](O)[C@H](O)[C@@H]1O)...,C35H39NO11,0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11829,DB16403,Isoquercitrin,investigational,[H][C@@]1(O[C@@H](OC2=C(OC3=C(C(O)=CC(O)=C3)C2...,C21H20O12,0100000000000000000000000000000000001000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11830,DB16404,Carvacrol,experimental,CC(C)C1=CC(O)=C(C)C=C1,C10H14O,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11831,DB16406,Subasumstat,investigational,CC1=C(C=C(S1)C(=O)C1=C(N[C@H]2C[C@H](O)[C@@H](...,C25H28ClN5O5S2,0000000001000000000010000000000100000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [57]:
drugdf.to_pickle("../dataset/drug/drug_full.pkl")

In [ ]:
#only approved

In [58]:
drugdf=pd.read_pickle("../dataset/drug/drug_full.pkl")
drugdf

,DrugBank ID,Name,Drug Groups,SMILES,Formula,Morgan Fingerprint,Morgan object
0,DB00006,Bivalirudin,approved; investigational,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33,0100000000000000000000000000000000000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,DB00007,Leuprolide,approved; investigational,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,DB00014,Goserelin,approved,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,DB00027,Gramicidin D,approved,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16,0100000000000000000000000000000001000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,DB00035,Desmopressin,approved,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
11828,DB16400,Sibofimloc,investigational,[H][C@@]1(O[C@H](CO)[C@@H](O)[C@H](O)[C@@H]1O)...,C35H39NO11,0000000000000000000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11829,DB16403,Isoquercitrin,investigational,[H][C@@]1(O[C@@H](OC2=C(OC3=C(C(O)=CC(O)=C3)C2...,C21H20O12,0100000000000000000000000000000000001000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11830,DB16404,Carvacrol,experimental,CC(C)C1=CC(O)=C(C)C=C1,C10H14O,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11831,DB16406,Subasumstat,investigational,CC1=C(C=C(S1)C(=O)C1=C(N[C@H]2C[C@H](O)[C@@H](...,C25H28ClN5O5S2,0000000001000000000010000000000100000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [59]:
drugdf=drugdf[drugdf['Drug Groups'].str.match('approved')]

In [60]:
drugdf

,DrugBank ID,Name,Drug Groups,SMILES,Formula,Morgan Fingerprint,Morgan object
0,DB00006,Bivalirudin,approved; investigational,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33,0100000000000000000000000000000000000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,DB00007,Leuprolide,approved; investigational,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,DB00014,Goserelin,approved,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,DB00027,Gramicidin D,approved,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16,0100000000000000000000000000000001000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,DB00035,Desmopressin,approved,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
11511,DB15934,Ferric glycerophosphate,approved; experimental,[Fe++].[Fe++].OCC(O)COP([O-])([O-])=O.OCCC(CCO...,C8H18Fe2O12P2,0100000000000000000000000000000000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11520,DB15950,Sodium camphorsulfonate,approved; experimental,[Na+].[H][C@]12CC[C@@](CS([O-])(=O)=O)(C(=O)C1...,C10H15NaO4S,0000000000000100000000000000000000000000000000...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
11550,DB15982,Berotralstat,approved,NCC1=CC(=CC=C1)N1N=C(C=C1C(=O)NC1=CC(=CC=C1F)[...,C30H26F4N6O,0100000000000000000000000000010000000000000000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11580,DB16019,Gallium Ga-68 gozetotide,approved,[H+].[68Ga+3].OC(=O)CC[C@H](NC(=O)N[C@@H](CCCC...,C44H59GaN6O17,0100000000000000000000000000000000000000010000...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [61]:
drugdf.to_pickle("../dataset/drug/approved_drug.pkl")